In [1]:
import tqdm
import json
from vowpalwabbit import pyvw

anchor_json = "wiki/nlwiki-20220301/experiments/clean-q0.25.json"

modelfile = "wiki/nlwiki-20220301/experiments/clean-q0.25.32b.vw"
datafile = "wiki/nlwiki-20220301/experiments/clean-q0.25.suriname.300.dat"
datafile = "wiki/nlwiki-20220301/experiments/suriname.parts/0.part"

surface_weights = json.load(open(anchor_json))
surface_weights = {
    m:{int(e.replace('Q','')):c for e,c in ec.items()} 
    for m, ec in surface_weights.items()
}

# audit = 'wiki/nlwiki-20220301/experiments/clean-q0.25.hash262144.utrecht.vw.audit'
# !rm $audit
model = pyvw.Workspace(
    initial_regressor=modelfile,
    loss_function="logistic",
    csoaa_ldf="mc",
    probabilities=True,
    
    testonly=True,
    data=datafile,
    predictions=datafile+'.pred',
    invert_hash=modelfile+'.inverted',
    # audit_regressor=modelfile+'.audit',
)
!grep -v '^:\?[0-9]' $modelfile".inverted" > $modelfile".inverted."
!mv $modelfile".inverted." $modelfile".inverted"
!rm $modelfile".inverted.writing"

creating quadratic features for pairs: ls
only testing
predictions = wiki/nlwiki-20220301/experiments/suriname.parts/0.part.pred
using no cache
Reading datafile = wiki/nlwiki-20220301/experiments/suriname.parts/0.part
num sources = 1
Num weight bits = 32
learning rate = 0.5
initial_t = 0
power_t = 0.5
Enabled reductions: gd, scorer-identity, csoaa_ldf-prob, shared_feature_merger
Input label = cs
Output pred = prob
average  since         example        example        current        current  current
loss     last          counter         weight          label        predict features
0.000000 0.000000            1            1.0          known            730     2160
0.000000 0.000000            2            2.0          known            730     2592
0.000000 0.000000            4            4.0          known            730     2000
0.000000 0.000000            8            8.0          known            730     2320
0.000000 0.000000           16           16.0          known            

[info] VW 9.0.0 introduced a change to the default model save behavior. Please use '--predict_only_model' when using either '--invert_hash' or '--readable_model' to get the old behavior. Details: https://vowpalwabbit.org/link/1


In [3]:
def read_instances(datafile):
    instance = []
    for line in open(datafile):
        line = line.strip()
        if not line:
            if len([l for l in instance if ':0' in l]) == 1:
                yield instance
            instance = []
        else:
            instance.append(line)
    if len([l for l in instance if ':0' in l]) == 1:
        yield instance

def predict_prob(model, instance):
    shared, candidates = instance[0], instance[1:]
    preds = {}
    for n in range(len(candidates)):
        l = int(candidates[n].split(':')[0])
        p = model.predict([shared] + candidates[n:] + candidates[:n])
        preds[l] = p
    return preds
        
for ex in read_instances(datafile):
    print(*ex, sep='\n')
    print('pred:', predict_prob(model, ex))
    break

shared |s surinaamse en andere regionale jeugdparlementariërs volgen 10 daagse training in china 20 juli 2017 surinaamse en andere regionale jeugdparlementariërs volgen 10 daagse training in china latijns amerikaanse en caribische jeugdparlementariërs hebben van 11 tot 20 juli het trainingsprogramma bridge tot the future in china gevolgd ook suriname vaardigde twee jeugdigen af het doel is de versterking van de onderlinge relatie het uitwisselen van kennis en het op verschillende gebieden invullen van bilaterale samenwerking china nam de organisatie en de kosten op zich van het nationaal jeugdparlement njp uit suriname namen kelvin konik voorzitter en abdul mohamed parlementslid deel aanvankelijk was minister joan dogojo van sport en jeugdzaken uitgenodigd zij vaardigde de njp leden af vanwege de begrotingsbehandeling en om de onderwerpen die aan de orde kwamen bronnen zelf schrijven hoe schrijf ik een artikel
730:0 |l suriname=730
7646305:1 |l suriname=7646305
327769:1 |l suriname=327

In [4]:
def read_pred(predfile):
    instance = []
    for line in open(predfile):
        line = line.strip()
        if not line:
            yield instance
            instance = []
        else:
            instance.append(float(line))
    yield instance

y_pred = []
for e,p in zip(read_instances(datafile), read_pred(datafile+'.pred')):
    p = max(zip(e[1:],p), key=lambda x: x[1])[0].split(':')[0]
    y_pred.append( int(p) )
len(y_pred)

34

In [5]:
# Resample to create actually balanced dataset

# inst = [tuple(i) for i in read_instances(datafile)]
# print(len(inst))

# df = pd.DataFrame({'i':inst, 'y':y_true}).merge((pd.Series(y_true).value_counts() > 100).rename('ok'), left_on='y', right_index=True)
# df = df[df['ok']].drop_duplicates().groupby('y').head(100).sample(frac=1)
# with open(datafile + '.reallybal', 'w') as fw:
#     for ex in list(df['i']):
#         for line in ex:
#             print(line, file=fw)
#         print(file=fw)

In [6]:
from collections import *
y_true = [int(x) for es in read_instances(datafile) for e in es if ':0' in e for x, y in [e.split(':',1)] if y.startswith('0')]
# y_pred = [model.predict(ex) for ex in read_instances(datafile)]
len(y_true), len(y_pred)

(34, 34)

In [20]:
import json
from minimel import normalize
from minimel.vectorize import vw_tok
lang = None

def model_predict(model, text, norm, ents, score=False):
        preds = {}
        ents = list(ents)
        toks = 'shared |s ' + ' '.join(vw_tok(text))
        ns = '_'.join(vw_tok(norm))
        for i,ent in enumerate(ents):
            cands = [f'{e} |l {ns}={e}' for e in ents[i:] + ents[:i]]
            print([toks] + cands)
            preds[ent] = model.predict([toks] + cands)
        if score:
            return preds
        else:
            return max(preds.items(), key=lambda x: x[1])[0]
        
# 
for line in open('wiki/nlwiki-20220301/experiments/mewsli-suriname.tsv'):
    _, surfaces, text = line.split('\t')
    for surface in json.loads(surfaces):
        for norm in normalize(surface, language=lang):
            if norm == 'suriname':
                ent_cand = surface_weights.get(norm, None)
                print( model_predict(model, text, norm, ent_cand) )
        break

['shared |s surinaams turks verdrag ter discussie 31 mei 2017 surinaams turks verdrag ter discussie in suriname liggen de betrekkingen met turkije onder een vergrootglas er is een nieuwe overeenkomst aanstaande die de samenwerking op meerdere terreinen moet gaan regelen in de overeenkomst worden thema behandeld als wederzijdse investeringen landbouw energie onderwijs gezondheidszorg toerisme en wetenschap en techniek in het debat lijkt de regeringsfractie ndp te gaan voor meer samenwerking met dit europees aziatische land andré misiekaba vraagt zich daarbij af waarom de 80 beschikbaar gestelde turkse studiebeurzen nooit zijn gebruikt dat zou volgens hem een diplomatieke blunder zijn voor de oppositie ligt de verdieping van de samenwerking met turkije gevoeliger die wijst op de verslechterde reputatie die turkije de afgelopen jaren in europa heeft opgebouwd tot openlijke internationale aanvaringen toe ook worden vragen over de voordelen gesteld en is het volgens chandrikapersad santokhi

0.000000 n.a.              512          512.0        unknown            730     1792


['shared |s bouterse voert druk op openbaar ministerie op juli 2017 bouterse voert druk op openbaar ministerie op desi bouterse president van suriname en hoofdverdachte in het proces van de decembermoorden uit 1982 voert de druk op het openbaar ministerie om op de auditeur militair van de krijgsraad eiste op 28 juni een onvoorwaardelijke celstraf van 20 jaar voor bouterse de kwaliteitsnieuwssite starnieuws meldt nu dat bouterse de procureur generaal pg roy baidjnath panday maande op te stappen in suriname heeft de pg de taak om verdachte politieke ambtsdragers te vervolgen zijn ambt is in principe voor het leven om te voorkomen dat de politiek invloed heeft op zijn functioneren panday zou bouterse om bedenktijd hebben gevraagd als reden van bouterse wordt gesteld dat panday onvoldoende zou hebben opgetreden tegen de president van de krijgsraad om het proces tegen hem te staken volgens bouterse zou de staatsveiligheid in gevaar zijn wanneer hij uit zijn ambt gezet zou worden het is maar

In [7]:
select_ents = set(y_true)

import pandas as pd
import sqlite3
from collections import *

con = sqlite3.connect('wiki/nlwiki-20220301/index_nlwiki-20220301.db')
ent_label = {}
for e in select_ents:
    l = pd.read_sql_query(f'select * from mapping where wikidata_id="Q{e}" limit 1', con)
    ent_label[e] = l['wikipedia_title'][0]

select_ents = dict(Counter(e for e in y_true if e in ent_label).most_common())    
dict(Counter(ent_label[e] for e in y_true if e in ent_label).most_common())

{'Suriname_(land)': 34}

In [8]:
from sklearn.metrics import classification_report
print('true', {ent_label.get(i):c for i,c in Counter(y_true).most_common(5)})
print('pred', {ent_label.get(i):c for i,c in Counter(y_pred).most_common(5)})
labels, target_names = zip(*{e:ent_label[e] for e in select_ents}.items())
print(classification_report(y_true, y_pred, labels=labels, target_names=target_names))

true {'Suriname_(land)': 34}
pred {'Suriname_(land)': 34}
                 precision    recall  f1-score   support

Suriname_(land)       1.00      1.00      1.00        34

       accuracy                           1.00        34
      macro avg       1.00      1.00      1.00        34
   weighted avg       1.00      1.00      1.00        34



In [9]:
fname = modelfile + '.inverted'
surface = 'suriname'

ls = [l.split()[0].split(':') for l in open(fname).readlines() if '*' in l and l.startswith(f'l^{surface}=')]
df = pd.DataFrame(ls)
ind = df[0].str[2:].str.split('*', expand=True)
ind[0] = ind[0].apply(lambda i: ent_label.get(int(i.split('=')[1])))
df = df[2].rename(0).astype('float').to_frame().set_index(pd.MultiIndex.from_frame(ind))

def topfeat(gr, neg=False):
    gr = gr.droplevel(0).rename(columns={0:'weight'})
    f = (lambda x: -x) if neg else 'abs'
    return gr.loc[gr['weight'].apply(f).sort_values().index[::-1]].head(10).sort_values('weight').reset_index()

df.groupby(level=0).apply(topfeat, neg=True).swaplevel().unstack().swaplevel(axis=1).sort_index(axis=1).T

0            1           2            3  \
0                                                                       
Suriname_(land) 1          s^is  s^categorie  s^suriname  s^politicus   
                weight -0.69764    -0.532371   -0.517769    -0.484377   

                                            4          5          6         7  \
0                                                                               
Suriname_(land) 1       s^verspreidingsgebied  s^ecuador  s^soorten  s^auteur   
                weight              -0.478954  -0.435425    -0.4138 -0.410344   

                               8           9  
0                                             
Suriname_(land) 1        s^omvat  s^volgende  
                weight -0.370743   -0.356981